In [3]:
!pip install pandas
!pip install scikit-learn

In [4]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import ModelCheckpoint
import keras
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.utils import to_categorical, plot_model

In [5]:
data_path = './dataset'
classes = [d for d in os.listdir(data_path)
           if os.path.isdir(os.path.join(data_path, d)) and d != '.ipynb_checkpoints']

print("Dataset Classes:")
for cls in classes:
    print(cls)

Dataset Classes:
Tomato___Bacterial_spot
Tomato___Early_blight
Tomato___healthy
Tomato___Late_blight
Tomato___Leaf_Mold
Tomato___Septoria_leaf_spot
Tomato___Spider_mites Two-spotted_spider_mite
Tomato___Target_Spot
Tomato___Tomato_mosaic_virus
Tomato___Tomato_Yellow_Leaf_Curl_Virus


In [6]:
def create_dataframe(data_path):
    filepaths = []
    labels = []

    for fold in os.listdir(data_path):
        f_path = os.path.join(data_path, fold)
        if os.path.isdir(f_path):
            imgs = os.listdir(f_path)
            for img in imgs:
                img_path = os.path.join(f_path, img)
                filepaths.append(img_path)
                labels.append(fold)

    fseries = pd.Series(filepaths, name='Filepaths')
    lseries = pd.Series(labels, name='Labels')
    return pd.concat([fseries, lseries], axis=1)

In [7]:
df = create_dataframe('./dataset')
print(df.head())
print("Jumlah total data:", len(df))

                                           Filepaths                   Labels
0  ./dataset\Tomato___Bacterial_spot\00416648-be6...  Tomato___Bacterial_spot
1  ./dataset\Tomato___Bacterial_spot\0045ba29-ed1...  Tomato___Bacterial_spot
2  ./dataset\Tomato___Bacterial_spot\00639d29-2d1...  Tomato___Bacterial_spot
3  ./dataset\Tomato___Bacterial_spot\00728f4d-83a...  Tomato___Bacterial_spot
4  ./dataset\Tomato___Bacterial_spot\00a7c269-347...  Tomato___Bacterial_spot
Jumlah total data: 18160


In [8]:
from sklearn.model_selection import train_test_split

# Split 80% train + 20% temp
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['Labels'], random_state=42)

# Dari 20% sisanya, split jadi 50% valid dan 50% test (jadi masing-masing 10% dari total)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['Labels'], random_state=42)

print(f"Train: {len(train_df)}")
print(f"Validation: {len(valid_df)}")
print(f"Test: {len(test_df)}")

Train: 14528
Validation: 1816
Test: 1816


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (299, 299)
batch_size = 32

train_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, rotation_range=20, zoom_range=0.2)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_dataframe(
    train_df, x_col='Filepaths', y_col='Labels',
    target_size=img_size, class_mode='categorical', batch_size=batch_size
)

test_data = test_gen.flow_from_dataframe(
    test_df, x_col='Filepaths', y_col='Labels',
    target_size=img_size, class_mode='categorical', batch_size=batch_size, shuffle=False
)

Found 14528 validated image filenames belonging to 10 classes.
Found 1816 validated image filenames belonging to 10 classes.


In [10]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
base_model.trainable = False  # agar lebih cepat saat training awal

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(len(train_data.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 73,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 73, 73,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 71, 71,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 71, 71,    │          0 │ batch_normalizat

 Total params: 22,066,346 (84.18 MB)

 Trainable params: 263,562 (1.01 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [11]:
checkpoint = ModelCheckpoint(
    filepath='best_model.h5',         # nama file model
    monitor='val_accuracy',           # metrik yang dipantau
    save_best_only=True,              # hanya simpan model terbaik
    mode='max',                       # karena semakin tinggi akurasi lebih baik
    verbose=1                         # tampilkan info saat model disimpan
)

In [13]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=50,
    verbose=1,
    callbacks=[early_stop]
)

Epoch 1/50
454/454 ━━━━━━━━━━━━━━━━━━━━ 551s 1s/step - accuracy: 0.4981 - loss: 1.5185 - val_accuracy: 0.7456 - val_loss: 0.8184
Epoch 2/50
454/454 ━━━━━━━━━━━━━━━━━━━━ 470s 1s/step - accuracy: 0.7450 - loss: 0.7853 - val_accuracy: 0.8194 - val_loss: 0.6063
Epoch 3/50
454/454 ━━━━━━━━━━━━━━━━━━━━ 468s 1s/step - accuracy: 0.7982 - loss: 0.6313 - val_accuracy: 0.8376 - val_loss: 0.5278
Epoch 4/50
454/454 ━━━━━━━━━━━━━━━━━━━━ 654s 1s/step - accuracy: 0.8176 - loss: 0.5602 - val_accuracy: 0.8502 - val_loss: 0.4741
Epoch 5/50
454/454 ━━━━━━━━━━━━━━━━━━━━ 487s 1s/step - accuracy: 0.8277 - loss: 0.5185 - val_accuracy: 0.8431 - val_loss: 0.4737
Epoch 6/50
454/454 ━━━━━━━━━━━━━━━━━━━━ 470s 1s/step - accuracy: 0.8418 - loss: 0.4842 - val_accuracy: 0.8695 - val_loss: 0.4180
Epoch 7/50
454/454 ━━━━━━━━━━━━━━━━━━━━ 473s 1s/step - accuracy: 0.8485 - loss: 0.4620 - val_accuracy: 0.8574 - val_loss: 0.4176
Epoch 8/50
454/454 ━━━━━━━━━━━━━━━━━━━━ 515s 1s/step - accuracy: 0.8578 - loss: 0.4325 - val_accu

In [14]:
model.save('./model/tomat_model.h5')

In [15]:
df.to_csv('./model/dataset.csv', index=False)

In [19]:
import tensorflow as tf

# Muat model dari file .h5
model = tf.keras.models.load_model('./model/tomat_model.h5')

# Konversi ke format TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan model TFLite ke file
with open('tomat_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpzrxsogjf\assets


INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpzrxsogjf\assets


Saved artifact at 'C:\Users\ASUS\AppData\Local\Temp\tmpzrxsogjf'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  1918486658320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1918486656400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1918486658704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1918486657360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1918486658896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1918486655632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1918486657552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1918486657168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1918486656208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1918486654864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  191848